# 🤖 Building Autonomous Agents: Adding Memory

Welcome to the first step in building autonomous agents! In this notebook, we'll focus on expanding an LLM's capabilities by giving it memory

LLMs are stateless meaning they have no idea about previous interactions. That means we need to pass in the context to the LLM each time. This is what we refer to as "memory". APIs like the converse API allow us to pass messages as JSON objects. Under the hood, it's converting that JSON into a conversation history that the LLM can understand.

## Objectives:
- Understand memory
- Add a simple sliding window memory implementation
- Introduce long term memory
- See how memory works in tools like LangChain.

Throughout this notebook, we'll use a simple ChatBot to start (based on module 1 notebook 2) and extend it to have memory.

Let's begin by setting up our environment and defining the chat bot 🚀

In [ ]:
# Import dependencies. 

import boto3
import json

# Initialize the Bedrock client
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')

print("✅ Setup complete!")

# The Case For Abstraction
If you remember in module 1, we built a basic chat bot. Let's create a new chat bot with conversational memory in vanilla Python code to understand the concept at their core.

An important design decision to make is how much you want to rely on a single GenAI framework. The more of the core types you own, the more flexible your system becomes. Want to switch from LangChain to Pydantic AI? Easy, just write code that converts their types into yours and the rest of the system doesn't care which framework you used. These types become abstraction layers between the framework and the rest of your code. (1) this is general coding best practice, but (2) it increases flexibility. The tradeoff is that you now own more code to maintain. If there's a major re-write between V1 and V2 of a framework and the types change, you'll spend a lot of time refactoring. But if it doesn't, then you've saved yourself some time and code. Because the space isn't mature, we suggest owning your own types for now.

All the major frameworks have their own implementation of session context / messages. However, we want to own our own types to create 2 way door decisions down the road that are reversable. Writing a converter is much simpler than refactoring an entire code base if you want to change your mind later. 

Let's start with a chat bot but extend it to use our own memory. First up is creating types for our messages and conversations. These types live in the agentic platform code (src/agent_platform). Lets look at a couple of them to see how we structured them


In [ ]:
from agentic_platform.core.models.memory_models import Message, SessionContext

In [ ]:
Message??

In [ ]:
SessionContext??

These are the main types we'll be playing with in the memory section of this notebook. As you can see, we've adoped a message and session context format mostly similar to other providers. 

The ChatCompletion response type has become very popular over the last two years. 

In 2025, it appears that the industry is shifting more to Anthropic (and Bedrock's converse) style output. OpenAI has the Response object which looks a lot more similar to Anthropic and Bedrock's types where content is a list of objects each containing a "type". Using content objects (or content blocks) makes streaming a bit easier too.

We've opted to make our object align more closely to where we "think" the industry is heading using the content list. However, we've added some helper functions like get_text_content() so we aren't iterating over a list every time we want to access the results of the message. This gives us the best of both worlds as we wait for specifications to solidify.

## Abstract LLM Touchpoints
We've typed everything up to this point with the exception of the LLM Request and LLM Response. Let's go ahead and output the response types we've created in the agentic platform

In [ ]:
from agentic_platform.core.models.llm_models import LLMRequest, LLMResponse

LLMRequest??

In [ ]:
LLMResponse??

## Writing Converters
Owning your types often means you need to write converters. This is a small price to pay for flexibility. Fortunately with modern AI coding assistants, writing converts is pretty painless

We've created a ConverseMessageConverter class you can use to do the conversion which we'll import below. This is pretty undifferentiated work so we left it out of the lab itself.

In [ ]:
# Import Converse API converter to convert the raw JSON into our own types.
from agentic_platform.core.converter.llm_request_converters import ConverseRequestConverter
from agentic_platform.core.converter.llm_response_converters import ConverseResponseConverter


# Call Bedrock Converse()
Now that we have our types and converters, we can simplify the Bedrock calls substantially by just passing in the request object and getting the response object back. The rest of the code base doesn't need to know any specifics of the API itself because it's all abstracted away

In [ ]:
# Helper function to call Bedrock. Passing around JSON is messy and error prone.
from typing import Dict, Any
def call_bedrock(request: LLMRequest) -> LLMResponse:
    kwargs: Dict[str, Any] = ConverseRequestConverter.convert_llm_request(request)
    # Call Bedrock
    converse_response: Dict[str, Any] = bedrock.converse(**kwargs)
    # Get the model's text response
    return ConverseResponseConverter.to_llm_response(converse_response)

# Create Memory Client
Lastly, we need to create a MemoryClient that can vend us conversations. In this implementation we'll be doing this locally. In a production environment you would want to swap out the memory client with one that calls a production grade database like DynamoDB or anyting else that works well with key/value pairs. 

Going back to the importance of abstraction, if your MemoryClient takes in a conversationId and returns a Conversation object, the rest of your code doesn't care what database, vendor, etc.. you're using. The important thing is to own your own types.

In [ ]:
class MemoryClient:
    """Manages conversations"""
    def __init__(self):
        self.conversations: Dict[str, SessionContext] = {}

    def upsert_conversation(self, conversation: SessionContext) -> bool:
        self.conversations[conversation.session_id] = conversation

        print(f'Conversation upserted: {self.conversations[conversation.session_id]}')

    def get_or_create_conversation(self, conversation_id: str=None) -> SessionContext:
        return self.conversations.get(conversation_id, SessionContext()) if conversation_id else SessionContext()
    
memory_client: MemoryClient = MemoryClient()

# Create Chat Bot With Memory
This is a very simple chat bot and there's no need for complex orchestration using LangGraph.

For this implementation, we'll keep it simple with vanilla Python

In [ ]:
# Import the same base prompt we've been using in the previous labs.
from agentic_platform.core.models.prompt_models import BasePrompt
from typing import Optional
from pydantic import BaseModel

# WE have two more agent types we'll incorporate from our platform. 
from agentic_platform.core.models.api_models import AgenticRequest, AgenticResponse

class MemoryAgentPrompt(BasePrompt):
    system_prompt: str = "You are a helpful assistant. Respond to the user as best as you can."
    user_prompt: str = "{user_message}"

class MemoryAgent:

    def __init__(self, prompt: BasePrompt):
        self.prompt = prompt

    def call_llm(self, context: SessionContext) -> LLMResponse:
        # Create LLM request
        request: LLMRequest = LLMRequest(
            system_prompt=self.prompt.system_prompt,
            messages=context.get_messages(),
            model_id=self.prompt.model_id,
            hyperparams=self.prompt.hyperparams
        )

        response: LLMResponse = call_bedrock(request)
        # Return the response.
        return response

    def invoke(self, request: AgenticRequest) -> AgenticRequest:
        # Get or create conversation
        context = memory_client.get_or_create_conversation(request.session_id)
        # Add user message to conversation. Using a convenience frunction from_text()
        # To create a message object with a content list of one text object.
        context.add_message(request.message)
        # Call the LLM.
        response: LLMResponse = self.call_llm(context)
        # Append the llms response to the conversation.
        response_msg: Message = Message.from_text(role="assistant", text=response.text)
        context.add_message(response_msg)
        # Save updated conversation
        memory_client.upsert_conversation(context)

        return AgenticResponse(
            message=response_msg,
            session_id=context.session_id
        )

# Call Chat Bot
Now that we have a chat bot, lets invoke it. Reiterating the importance of abstraction, we'll create our own Message type and convert it 

In [ ]:
# Helper to construct request
def construct_request(user_message: str, conversation_id: str=None) -> AgenticRequest:
    return AgenticRequest.from_text(
        text=user_message, 
        **{'session_id': conversation_id}
    )

agent: MemoryAgent = MemoryAgent(MemoryAgentPrompt()) 

# Invoke the agent
request: AgenticRequest = construct_request("Hello!")
print(request)
agent.invoke(request)

Now lets test it with a multi-turn conversation. The first response will return a conversationId. In the second turn, we'll pass in that conversation Id to let our code know it needs to send in the past conversation as well. 

In [ ]:
agent: MemoryAgent = MemoryAgent(MemoryAgentPrompt()) 

user_message: str = "Hello, can you quickly tell me why the sky is blue? One sentence is fine."
request: AgenticRequest = construct_request(user_message)

response: AgenticRequest = agent.invoke(request)

print(response)

Great! We have a response and now a new conversation_id as well. Lets check out what the conversation history looks like in our memory "store"

In [ ]:
conversation: SessionContext = memory_client.get_or_create_conversation(response.session_id)
from pydantic_core import to_jsonable_python

# Use the pydantic model_dump_json. The content list uses a base class so we need to serialize as any to see specific subclass attributes.
print(conversation.model_dump_json(indent=2, serialize_as_any=True))

Now lets test with multi turn. We'll pass in the conversationId from the response object and ask the model what we were talking about about previously. The response should tell us that we were asking why the sky is blue

In [ ]:
conversation_id: str = response.session_id

# Construct a new request with the conversationId
request: AgenticRequest = construct_request('What were we talking about again?', conversation_id)

# Invoke the agent
response: AgenticRequest = agent.invoke(request)
print('--- Models response ---')
print(response.model_dump_json(indent=2, serialize_as_any=True))

# What did we just do?
Congrats! We successfully added short term memory to our agent! This allows conversations to flow more freely and give us our first LLM augmented component on our path to building an agent!

Next, we'll explore how to add tool use to this agent 🚀
